In [1]:
! which python && which pip

/home/buka2004/PTQ-LLM-MIPT/.venv10/bin/python
/home/buka2004/PTQ-LLM-MIPT/.venv10/bin/pip


# Set python path

In [2]:
! export PYTHONPATH="/home/buka2004/PTQ-LLM-MIPT:$PYTHONPATH"

# Import libs

In [3]:
from llmcompressor.transformers import oneshot
from llmcompressor.modifiers.quantization import GPTQModifier
from llmcompressor.modifiers.smoothquant import SmoothQuantModifier
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
from datasets import load_dataset
from deepspeed.compression.helper import convert_conv1d_to_linear
from transformers.modeling_utils import Conv1D

/home/buka2004/PTQ-LLM-MIPT/.venv10/lib/python3.10/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlvsym'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlopen'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlclose'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlerror'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlsym'
collect2: error: ld returned 1 exit status


# Test vllm smoothquant quantization

In [4]:
from llmcompressor.modifiers.smoothquant import SmoothQuantModifier
from llmcompressor.modifiers.quantization import GPTQModifier
from llmcompressor import oneshot

# Select quantization algorithm. In this case, we:
#   * apply SmoothQuant to make the activations easier to quantize
#   * quantize the weights to int8 with GPTQ (static per channel)
#   * quantize the activations to int8 (dynamic per token)
recipe = [
    SmoothQuantModifier(smoothing_strength=0.8),
    GPTQModifier(scheme="W8A8", targets="Linear", ignore=["lm_head"]),
]

# Apply quantization using the built in open_platypus dataset.
#   * See examples for demos showing how to pass a custom calibration set
output_dir='/home/buka2004/PTQ-LLM-MIPT/notebooks/llm_out/TinyLlama-1.1B-Chat-v1.0-INT8'

oneshot(
    model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    dataset="open_platypus",
    recipe=recipe,
    output_dir=output_dir,
    max_seq_length=2048,
    num_calibration_samples=512,
    log_dir=output_dir
)

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Cancellation requested; stopping current tasks.


KeyboardInterrupt: 